In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
import csv
from batchmake import Batcher
import pprint
tf.set_random_seed(777)
from tensorflow.python.layers import core

In [2]:
datas= 'data_samples/'
vocabulary = 'vocab.csv'
vocabs = []
with open(vocabulary, 'r', newline='', encoding='utf-8') as vocab:
    words = csv.reader(vocab)
    for i, word in enumerate(words):
        vocabs.append(word[0])
#         vocabs[word[0]]=i
#         vocabs[word] = i
print(len(vocabs))
word_to_id = {word: i for i, word in enumerate(vocabs)}
id_to_word = {i:word for i, word in enumerate(vocabs)}

22673


In [3]:
id_to_word[0], word_to_id['<PAD>']

('<PAD>', 0)

In [4]:
hidden_size=32
sequence=5
embedding_dim=50
attention_size = 50
batch_size=16
vocab_size=len(vocabs)

In [5]:
def get_model_inputs():
    X = tf.placeholder(tf.int32, [None, sequence], name='inputs_xdata')
    # Y = tf.placeholder(tf.float32, [None, sequence], name='targets_ydata')
    Y = tf.placeholder(tf.int32, [None, sequence], name='targets_ydata')
#     seq_len = tf.placeholder(tf.int32, [None], name='seq_len')
    lr = tf.placeholder(tf.float32, name='learning_rate')

#     embedding_variable = tf.Variable(tf.random_uniform([vocab_size, embedding_dim],-1.0,1.0), trainable=True)
#     batch_embedded = tf.nn.embedding_lookup(embedding_variable, X)
    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    
    return X, Y, lr, target_sequence_length, source_sequence_length, max_target_sequence_length

In [6]:
def encoding_layer(input_data, rnn_size, num_layers,
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    # Encoder embedding
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, encoding_embedding_size)

    # RNN cell
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return enc_cell

    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell, enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return enc_output, enc_state

In [7]:
def process_decoder_input(target_data, word_to_int, batch_size):
    ending = tf.strided_slice(target_data, [0,0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size,1], word_to_int['<GO>']), ending], 1)
    return dec_input

In [8]:
def decoding_layer(target_letter_to_int, decoding_embedding_size, num_layers, rnn_size,
                   target_sequence_length, max_target_sequence_length, enc_state, dec_input):
    
    target_vocab_size = len(target_letter_to_int)
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])

    output_layer = core.Dense(target_vocab_size,
                              kernel_initializer=tf.truncated_normal_initializer(mean = 0.0, stddev=0.1)
                             )
    
    with tf.variable_scope('decode'):
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                            sequence_length = target_sequence_length,
                                                            time_major=False
                                                           )
        
        training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                           training_helper,
                                                           enc_state,
                                                           output_layer
                                                          )
        
        training_decoder_output,final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                                                        impute_finished=True,
                                                                                                        maximum_iterations=max_target_sequence_length
                                                                                                       )
        
    with tf.variable_scope('decode', reuse=True):
        start_tokens = tf.tile(tf.constant([target_letter_to_int['<GO>']],dtype=tf.int32), [batch_size], name='start_tokens')
        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                                    start_tokens,
                                                                    target_letter_to_int['<EOS>']
                                                                   )
        
        inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                                            inference_helper,
                                                            enc_state,
                                                            output_layer
                                                           )
        
        inference_decoder_output, final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                                                                          impute_finished=True,
                                                                                                          maximum_iterations=max_target_sequence_length
                                                                                                         )
        
        print('training_decoder_output.rnn_output, inference_decoder_output.sample_id', training_decoder_output.rnn_output, inference_decoder_output.sample_id)
    return training_decoder_output, inference_decoder_output

In [9]:
def seq2seq_model(input_data, targets, target_sequence_length,max_target_sequence_length,
                  source_sequence_length,source_vocab_size, target_vocab_size, enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers
                 ):

    _, enc_state = encoding_layer(input_data=input_data, rnn_size=rnn_size, num_layers=num_layers,
                                  source_sequence_length = source_sequence_length,
                                  source_vocab_size = source_vocab_size,
                                  encoding_embedding_size=embedding_dim
                                 )
    
    dec_input = process_decoder_input(targets, word_to_id, batch_size)

    training_decoder_output, inference_decoder_output = decoding_layer(target_letter_to_int=word_to_id,
                                                                       decoding_embedding_size=embedding_dim,
                                                                       num_layers = num_layers,
                                                                       rnn_size=rnn_size,
                                                                       target_sequence_length=target_sequence_length,
                                                                       max_target_sequence_length=max_target_sequence_length,
                                                                       enc_state=enc_state,
                                                                       dec_input=dec_input
                                                                      )
    
    return training_decoder_output,inference_decoder_output

In [10]:
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():

    # Load the model inputs    
    input_data, targets, lr, target_sequence_length, source_sequence_length, max_target_sequence_length = get_model_inputs()
    print('input_data fetched ')
    # Create the training and inference logits
    training_decoder_output, inference_decoder_output = seq2seq_model(input_data=input_data, 
                                                                      targets=targets, 
                                                                      target_sequence_length = target_sequence_length, 
                                                                      max_target_sequence_length = max_target_sequence_length, 
                                                                      source_sequence_length = source_sequence_length,
                                                                      source_vocab_size = len(word_to_id),
                                                                      target_vocab_size = len(word_to_id),
                                                                      enc_embedding_size = embedding_dim, 
                                                                      dec_embedding_size = embedding_dim, 
                                                                      rnn_size = hidden_size, 
                                                                      num_layers = 1)
    
    print('seq2seq model created! ')
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_decoder_output.rnn_output, 'logits')
    inference_logits = tf.identity(inference_decoder_output.sample_id, name='predictions')

    # Create the weights for sequence_loss
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):

        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(training_logits, targets, masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
        print('optmization created!')

input_data fetched 
training_decoder_output.rnn_output, inference_decoder_output.sample_id Tensor("decode/decoder/transpose:0", shape=(16, ?, 22673), dtype=float32) Tensor("decode_1/decoder/transpose_1:0", shape=(16, ?), dtype=int32)
seq2seq model created! 
optmization created!


In [11]:
# Ses = tf.Session(graph=train_graph)
with tf.Session(graph=train_graph) as Ses:
    print('session !')
    file_name_queue = tf.train.string_input_producer(['inputs'])
    reader = tf.TFRecordReader()
    _, single_x = reader.read(file_name_queue)
    print('gogo~')
    context_features = {
        "length": tf.FixedLenFeature([], dtype=tf.int64)
    }

    sequence_features = {
        "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64),
        "labels": tf.FixedLenSequenceFeature([], dtype=tf.int64)
    }

    context_parsed, sequence_parsed = tf.parse_single_sequence_example(serialized=single_x,
                                                                       context_features=context_features,
                                                                       sequence_features=sequence_features
                                                                      )
    context_parsed['length'] = tf.cast(context_parsed['length'], tf.int32)
    batch_seq, batch_inputs = tf.contrib.training.bucket_by_sequence_length(context_parsed['length'],
                                                                            [sequence_parsed['tokens'], sequence_parsed['labels']],
                                                                            batch_size=batch_size,
                                                                            bucket_boundaries=[5,10],
                                                                            dynamic_pad=True,
                                                                            capacity=1000,
                                                                            num_threads=1
                                                                           )

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=Ses, coord=coord)
    Ses.run(tf.global_variables_initializer())
    epochs = 3
    for epoch in range(1, epochs):
        print('epoch', epoch)
        i = 0
        while True:

            seq_input_, batch_input_ = Ses.run([batch_seq, batch_inputs])

            feed_data = {input_data: batch_input_[0], targets: batch_input_[1], 
                         target_sequence_length:seq_input_, source_sequence_length:seq_input_, lr:0.1}
            _, loss = Ses.run([train_op, cost], feed_data)
    #             print('started')
            print(loss)
            i+=1
            print(i)
            if i % 50 ==0:
                checkpoint = './seq2seq_test/seq2seq_test.ckpt'
                saver = tf.train.Saver()
                saver.save(Ses, checkpoint)
                print('model saved')

session !
gogo~
epoch 1
10.0463
1
8.95776
2
7.11115
3
6.50275
4
5.27128
5
5.59503
6
6.07067
7
6.81945
8
6.3651
9
6.10921
10
6.76331
11
6.0456
12
6.62094
13
6.53642
14
6.30824
15
6.77583
16
6.20175
17
5.54972
18
5.22091
19
6.87305
20
4.51252
21
5.29352
22
5.98004
23
5.81741
24
5.17061
25
5.00275
26
5.34604
27
7.83054
28
5.80817
29
4.68187
30
4.31279
31
6.74574
32
8.18326
33
7.03924
34
5.87046
35
6.01217
36
4.39873
37
4.59124
38
4.99657
39
6.01691
40
5.83225
41
4.81226
42
4.16004
43
3.92675
44
3.96067
45
4.10784
46
4.05394
47
4.00941
48
4.3141
49
4.12576
50
model saved
6.36031
51
7.35015
52
7.25151
53
6.38877
54
6.01545
55
5.1642
56
5.26874
57
4.3124
58
4.7563
59
4.66316
60
4.70267
61
5.04084
62
5.12119
63
4.83196
64
4.30262
65
5.0223
66
5.09584
67
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], 

KeyboardInterrupt: 